#### 필요 라이브러리 Import 

In [6]:
from selenium import webdriver
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import time
import re
import unicodedata
import requests

#### 드라이버 기본 설정 

In [79]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option("excludeSwitches", ["enable-logging"])

driver = webdriver.Chrome("./chromedriver.exe", options=options)

#### Instagram 접속 

In [80]:
driver.get('https://www.instagram.com')   
time.sleep(2)

#### Instagram login

In [81]:
email = input('이메일을 입력하세요.')
pw = input('비밀번호를 입력하세요. 결과 창에 비밀번호가 공개되는 것을 방지하려면 아무것도 입력하지 마세요.\
            생성된 크롬창에서 직접 비밀번호를 입력하세요.')

# id 입력 
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

# pw 입력 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(pw)
input_pw.submit()

print('인스타그램 로그인 확인을 위한 10초 타이머 시작')
for i in tqdm(range(10)):   # 로그인 확인을 위한 대기시간 10초 
    time.sleep(1)

이메일을 입력하세요.pomate@g.skku.edu
비밀번호를 입력하세요. 결과 창에 비밀번호가 공개되는 것을 방지하려면 아무것도 입력하지 마세요.            생성된 크롬창에서 직접 비밀번호를 입력하세요.Dhwoehd12
인스타그램 로그인 확인을 위한 10초 타이머 시작


  0%|          | 0/10 [00:00<?, ?it/s]

#### 인스타 계정 검색 

In [10]:
def insta_searching(insta_id):
    '''
    검색하고 싶은 해시태그를 입력으로 받아, url을 반환
    '''
    url = 'https://www.instagram.com/' + insta_id
    return url

#### 계정 정보 크롤링하기 
- 해당 코드를 모두 실행하면 2개 파일이 생성됨 
- 계정 자체 정보를 가지고 있는 csv 파일 
- 계정의 최근 9개 게시물 정보를 가지고 있는 csv 파일

In [35]:
def select_first(driver):
    '''
    해당 계정의 게시글 중 첫번째 게시물을 선택하는 함수
    '''
    for idx in range(5):
        try:
            driver.find_element_by_xpath(f'//*[@id="react-root"]/div/div/section/main/div/div[{idx}]\
            /article/div[1]/div/div[1]/div[1]/a/div[1]/div[2]').click()
        except:
            continue
    time.sleep(3)

In [67]:
def get_account_info(driver):
    '''
    계정 정보 크롤링 
    계정 ID, 게시글 수, 팔로워 수를 크롤링해서 반환
    '''
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    try:   # 계정 ID 정보 가져오기
        account = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/section/main/div/header/section/div[1]/h2').text 
        print(account)
    except:
        account = ''
    try:   # 게시글 수 가져오기 
        num_contents = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/section/main/div/header/section/ul/li[1]/div/span').text 
    except:
        num_contents = 0
    try: 
        num_followers = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/section/main/div/header/section/ul/li[2]/a/div/span').text
    except:
        num_followers = 0 
    data = [account, num_contents, num_followers]
    return data 

In [70]:
def get_content(driver):
    '''
    크롤링할 콘텐츠 지정 
    본문 내용, 좋아요 수, 위치정보, 해시태그, 댓글 수를 크롤링해서 반환  
    '''
    # 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    try: 
        account = driver.find_elemnet_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[1]/div/header/div[2]/div[1]/div[1]/div[1]/span/a').text 
    except:
        account = ''
    try:   # 본문 내용 가져오기
        content = driver.find_element_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]\
        /ul/div/li/div/div/div[2]/div[1]/span').text 
        print(content)
        # content = unicodedata.normalize('NFC', content) 
    except:
        content = ''
    # 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    try:   # 좋아요 수 가져오기  
        like = driver.find_element_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/\
        div/div/div/a/div/span').text    
    except:
        like = 0
    try:   # 위치정보 가져오기
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    try:   # 댓글 수 가져오기
        flag = True 
     
        while flag:
            try: 
                driver.find_element_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/li/div/button').click()
                time.sleep(2)
            except:
                flag = False
                
        for idx in range(1, 99999):
            try:  # 댓글 정보 수집 
                comment = driver.find_element_by_xpath(f'/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[{idx}]/div/li/div/div/div[2]/div[1]/span').text
                # print(comment)
            except:  # 댓글 수집이 불가한 경우 
                print(idx)
                break
        num_comment = idx 
    except:
        num_comment = 0 
    # 수집한 정보 저장하기
    data = [account, content, like, place, tags, num_comment]
    return data

In [74]:
def account_crawling(driver):
    '''
    계정 id를 입력받아 해당 계정의 id, 게시글 수, 팔로워 수 정보(account_info)와
    해당 계정의 최근 9개 게시글에 관한 정보(content_info) 반환 
    '''
    word = input('수집할 계정 id를 입력해주세요: ')    # 검색어
    url = insta_searching(word)
    driver.get(url)
    time.sleep(5)
    
    account_info = get_account_info(driver)
    content_info = [] 
    select_first(driver)
    time.sleep(3)
    
    # 최근 9개 정보 수집 
    for idx in tqdm(range(1, 10)):
        try:
            data = get_content(driver)    # 게시글 정보 가져오기
            content_info.append(data)
            if idx == 1:
                driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div/button').click()
                time.sleep(1)
            else:
                driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div[2]/button').click()
                time.sleep(1)
        except:
            continue 
    
    return account_info, content_info

In [82]:
a_info, c_info = account_crawling(driver)

수집할 계정 id를 입력해주세요: seoul_official
seoul_official


  0%|          | 0/9 [00:00<?, ?it/s]

#서울플레이리스트 🌺
지금은 튤립과 라일락의 시간,
다섯번째 트랙 “로맨틱 봄날🌷”

#사랑을전하는서울 #💗
#봄날 #튤립 #라일락 #로맨틱

※ 매주 목요일마다 인스타그램 속 #서울의봄 풍경을 발굴해 소개해 드리는 #서울플레이리스트 시리즈!
다음 주는 어린이 날로 한 주 쉬어갑니다 👶
5월 둘째 주 목요일에 만나요~ :-)

👇사진 속 장소는👇
📷 1, 2, 6. #연희숲속쉼터 #Ansan Mountain
📷 3, 5. #서울숲 #SeoulForest
📷 4, 7. #중랑천 #Jungnangcheon Stream
📷 8. #회현동 (#연세대학교세브란스빌딩 ) #Hoehyeondong
📷 9. #창동 #Changdong
📷 10. #합정동 #Hapjeongdong

#나와서울
#seoul_official
18
수변테라스에서 커피 마시는 갬성..☕
도림·정릉·홍제천 대변신 예고!😎

📣#서울형_수변감성도시 사업 본격 추진! 실개천과 소하천 등을 중심으로 단순히 물리적 구조 정비가 아닌, 수변의 감성을 느끼면서 다양한 야외활동이 가능하도록 서울 곳곳을 매력적인 수(水)세권으로 재편합니다.
내년 상반기까지 '도림천' '정릉천' '홍제천(상·중류)'에서의 시범사업을 시작으로, 2030년까지 서울 전역으로 확대할 예정이에요🙂

🌊#도림천
신원시장‧순대타운 등 지역상권과 가까운 ‘도림천’은 음식과 문화를 함께 즐길 수 있도록 수변테라스 등을 조성할 계획!

🌊#정릉천
경관을 해치고 이용도도 떨어졌던 애물단지 복개구조물이 '도심 속 문화캔버스'를 콘셉트로 한 복합문화공간으로 재탄생 예정!

🌊#홍제천
홍제천 상류에는 수려한 암반 경관과 역사 문화재인 홍지문‧탕춘대성과 연계한 공간으로 조성, 중류인 인공폭포 주변에는 '노천카페' 오픈 예정!

#seoul_official
39
지금 청계천은?🌛✨

[연등회 #전통등전시회]
✨4월 26일 ~ 5월 10일까지 점등
📍청계천 입구 ~ 삼일교

#봄밤 #서울 #청계천 #청계광장 #연등회
#Cheonggyecheon #LotusLan